This notebook contains some consistency checks. Using the bias as an example, outliers in terms of especially high and low bias are plotted. One plot contains a map of the domain to see if the outliers are concentrated on a specific region. In another plot, time series of these special grid points are plotted to get a feeling for what is going on.

In [ ]:
import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt

# Load Configuration

In [ ]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
ds_ref = xr.open_zarr(CONFIG["path_ec_euro"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))
ds_mod = xr.open_zarr(CONFIG["path_xgb_v1"]).sel(time=slice("2021-01-01T00", "2022-11-30T00"))

# Bias

In [ ]:
fname = f"/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/spatial/bias.zarr"
ds_bias = xr.open_zarr(fname)

In [ ]:
#var = "swvl1"
var = "stl1"
#var = "snowc"

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set(title=f"Bias {var}")

vmin = np.nanpercentile(ds_bias.sel(variable=var).data, 1, axis=0)
vmax = np.nanpercentile(ds_bias.sel(variable=var).data, 99, axis=0)
im = ax.scatter(ds_bias["lon"], ds_bias["lat"], c=ds_bias.sel(variable=var).data, s=10, vmin=vmin, vmax=vmax)
fig.colorbar(im)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

l_p, u_p = 1, 99

lower_perc = np.nanpercentile(ds_bias.sel(variable=var).data, l_p, axis=0)
ax.axvline(lower_perc, color="tab:grey", ls="dashed")
ax.text(lower_perc, 0.99, f"{l_p}%", color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

upper_perc = np.nanpercentile(ds_bias.sel(variable=var).data, u_p, axis=0)
ax.axvline(upper_perc, color="tab:grey", ls="dashed")
ax.text(upper_perc, 0.99, f"{u_p}%", color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

ax.hist(ds_bias.sel(variable=var).data, bins=100)
plt.show()

In [ ]:
ii_below = np.where(ds_bias.sel(variable=var).data < lower_perc)[0]
ii_above = np.where(ds_bias.sel(variable=var).data > upper_perc)[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set(title=f"Smoothed anomalies of {var} with high negative bias")

anomalies = ds_mod.sel(variable=var).data.isel(x=ii_below) - ds_ref.sel(variable=var).data.isel(x=ii_below)
ax.plot(anomalies.rolling(time=7).mean()["time"], anomalies.rolling(time=7).mean())

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set(title=f"Smoothed anomalies of {var} with high positive bias")

anomalies = ds_mod.sel(variable=var).data.isel(x=ii_above) - ds_ref.sel(variable=var).data.isel(x=ii_above)
ax.plot(anomalies.rolling(time=7).mean()["time"], anomalies.rolling(time=7).mean())

plt.show()

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
fig, ax = plt.subplots(figsize=(10,4), subplot_kw={'projection': ccrs.PlateCarree()})

ax.set_extent([ds_ref["lon"].min(), ds_ref["lon"].max(), 
               ds_ref["lat"].min(), ds_ref["lat"].max()], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

ax.scatter(ds_ref.clim_data.isel(x=ii_above)["lon"], 
           ds_ref.clim_data.isel(x=ii_above)["lat"], 
           #c=ds_bias.data.sel(variable=var).isel(x=ii_above).values, 
           color="red", marker="x", s=80)

plt.show()

In [ ]:
i_worst = np.argmax(ds_bias.data.sel(variable=var).values)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D


def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            """ Draw. If frame is polygon, make gridlines polygon-shaped """
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)


        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)


                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

ranks = [(ds_ref.clim_data.isel(x=i_worst).sel(clim_variable=clim_var).values > 
          ds_ref.clim_data.sel(clim_variable=clim_var).values).mean() for clim_var in ds_ref.clim_variable.values]
    
data = [list(ds_ref.clim_variable.values),
        ('Title', [ranks])]

N = len(data[0])
theta = radar_factory(N, frame='polygon')

spoke_labels = data.pop(0)
title, case_data = data[0]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='radar'))
fig.subplots_adjust(top=0.85, bottom=0.05)

ax.set_rgrids([0.2, 0.4, 0.6, 0.8])
ax.set_title(title,  position=(0.5, 1.1), ha='center')

for d in case_data:
    line = ax.plot(theta, d)
    ax.fill(theta, d,  alpha=0.25)
ax.set_varlabels(spoke_labels)

plt.show()

## Defect grid point in ECLand?

In [ ]:
i = 2982

In [ ]:
for var in np.intersect1d(ds_mod.variable, ds_ref.variable):
    plt.plot(ds_ref.data.sel(variable=var).isel(x=i))
    plt.title(f"Variable {var}")
    plt.show()